<a href="https://colab.research.google.com/github/icecat14159/PL-Repo./blob/main/%E8%BF%BD%E7%95%AA%E6%B8%85%E5%96%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread #GoogleSheet

In [ ]:
!pip install gradio

In [ ]:
!pip install pandas #資料處理

In [3]:
from google.colab import auth #身分驗證
from google.auth import default #憑證
import gspread
import gradio as gr
import pandas as pd
import datetime #日期
import io #io

In [4]:
# 授權連線 Google Sheet
auth.authenticate_user()  #要求授權
creds, _ = default()  #獲取憑證
gc = gspread.authorize(creds) #授權

In [30]:
# 連結試算表
sheet_url = "https://docs.google.com/spreadsheets/d/1fGjPVqPHt3flo-LxBNU9EZl_ZMvg-I8UXfMhIY-jDNA/edit?usp=sharing"  #GoogleSheet連結
sh = gc.open_by_url(sheet_url)  #進入試算表
WORKSHEETS = {
    "novel": sh.worksheet("小說清單"),
    "comic": sh.worksheet("漫畫清單"),
    "anime": sh.worksheet("動畫清單")
}

In [49]:
# 讀取試算表
COLUMNS = ["ID", "作品名稱", "作者", "評級", "進度", "狀態", "最後紀錄日期"]  #資料庫7欄
DISPLAY_COLUMNS = COLUMNS + ["距離上次紀錄(天)"] #顯示用8欄
def read_data(media):
  worksheet = WORKSHEETS[media]
  records = worksheet.get_all_records()
  df = pd.DataFrame(records)

  if df.empty:
    df = pd.DataFrame(columns=COLUMNS)

  #確保ID
  if "ID" in df.columns:
    df["ID"] = pd.to_numeric(df["ID"], errors="coerce")

  #計算距離上次紀錄時間
  today = datetime.date.today()

  def calc_days(date_str):
    try:
      last_date = datetime.datetime.strptime(date_str, "%Y/%m/%d").date()
      return (today - last_date).days
    except:
      return None
  df["距離上次紀錄(天)"] = df["最後紀錄日期"].apply(calc_days)

  return df
# read_data(media)

In [50]:
# 寫回試算表
def write_data(df, media):
  worksheet = WORKSHEETS[media]
  df = df[COLUMNS] #強制只留下資料庫欄位
  worksheet.clear()
  worksheet.append_row(COLUMNS)
  worksheet.append_rows(df.values.tolist())
# write_data(read_data())

In [51]:
# 新增紀錄
def add_record(name, author, rating, progress, condition, date, media):
  worksheet = WORKSHEETS[media]
  df = read_data(media)
  if not date:
      date = datetime.date.today().strftime("%Y/%m/%d")
  if df.empty or df["ID"].dropna().empty: #產生ID
    new_id = 1
  else:
    new_id = int(df["ID"].max()) + 1
  new_entry = {"ID": new_id, "作品名稱": name, "作者": author, "評級": rating, "進度": progress, "狀態": condition, "最後紀錄日期": date}
  df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)
  write_data(df, media)
  return df

# add_record(name="測試作品A", author="測試作者", rating="B", progress="第1集", condition="未完結", date="")

In [52]:
# 修改紀錄
def edit_record(record_id, name, author, rating, progress, condition, date, media):
  worksheet = WORKSHEETS[media]
  df = read_data(media)
  if record_id not in df["ID"].values:
    return "找不到此 ID"
  if not date:
    date = datetime.date.today().strftime("%Y/%m/%d")

  df.loc[df["ID"] == record_id, ["作品名稱", "作者", "評級", "進度", "狀態", "最後紀錄日期"]] = \
      [name, author, rating, progress, condition, date]
  write_data(df, media)
  return df
# edit_record(1, "作品A-修正", "作者A", "A", "30", "未完結", "")

In [53]:
# 刪除紀錄
def delete_record(record_id, media):
  worksheet = WORKSHEETS[media]
  df = read_data()
  if record_id not in df["ID"].values:
    return "找不到此 ID"
  df = df[df["ID"] != record_id]
  write_data(df, media)
  return df
# delete_record(2)

In [54]:
# gradio用 改動表單
def save_table(table_df, media):
  worksheet = WORKSHEETS[media]
  df = pd.DataFrame(table_df, columns=DISPLAY_COLUMNS) # 轉成DataFrame並強制欄位順序
  df = df[COLUMNS] #丟掉第8欄
  df["ID"] = range(1, len(df) + 1) #修正ID（防止使用者亂改/刪列）

  today = datetime.date.today().strftime("%Y/%m/%d")
  df["最後紀錄日期"] = df["最後紀錄日期"].replace("", today)
  write_data(df, media)

  return read_data(media)

In [55]:
# gradio用 新增資料列
def add_empty_row(table_df, media):
  worksheet = WORKSHEETS[media]
  df = pd.DataFrame(table_df, columns=DISPLAY_COLUMNS)

  new_row = {
    "ID": "",
    "作品名稱": "",
    "作者": "",
    "評級": "",
    "進度": "",
    "狀態": "",
    "最後紀錄日期": "",
    "距離上次紀錄(天)": ""
  }

  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
  return df

In [61]:
# gradio用 刪除資料列
def delete_and_refresh(record_id, media):
  #防呆ID
  if record_id is None:
    return read_data(media)
  try:
    record_id = int(record_id)
  except:
    return read_data(media)

  df = read_data(media)

  if record_id not in df["ID"].values:
    return df

  # 執行刪除
  df = df[df["ID"] != record_id].reset_index(drop=True)

  # 防呆整表被清空
  if df.empty:
    return df

  #重排ID
  df["ID"] = range(1, len(df) + 1)

  write_data(df, media)
  return read_data(media)

In [66]:
#示警清單
ALERT_RULES = {
  "novel": 90, #小說3個月
  "comic": 30, #漫畫1個月
  "anime": 14 #動畫14天
}
ALERT_COLUMNS = [
    "ID",
    "作品名稱",
    "作者",
    "進度",
    "狀態",
    "最後紀錄日期",
    "距離上次紀錄(天)"
]

def get_alert_table(media):
  df = read_data(media)

  if df.empty:
      return df

  threshold = ALERT_RULES[media]

  alert_df = df[
      (df["狀態"] == "未完結") &
      (df["距離上次紀錄(天)"].notna()) &
      (df["距離上次紀錄(天)"] >= threshold)
  ]

  return alert_df.reset_index(drop=True)

In [74]:
# 跨媒體偵測
CROSS_MEDIA_CONFIG = {
    "novel": {
        "targets": ["comic", "anime"],
        "columns": ["對應漫畫ID", "對應動畫ID"]
    },
    "comic": {
        "targets": ["novel", "anime"],
        "columns": ["對應小說ID", "對應動畫ID"]
    },
    "anime": {
        "targets": ["novel", "comic"],
        "columns": ["對應小說ID", "對應漫畫ID"]
    }
}
def get_cross_columns(media):
  return ["ID", "作品名稱"] + CROSS_MEDIA_CONFIG[media]["columns"]

def build_cross_media_table(media):
  base_df = read_data(media)

  if base_df.empty:
      return pd.DataFrame(columns=get_cross_columns(media))

  targets = CROSS_MEDIA_CONFIG[media]["targets"]
  target_dfs = {t: read_data(t) for t in targets}

  rows = []

  for _, row in base_df.iterrows():
      title = row["作品名稱"]
      base_id = int(row["ID"])

      cross_ids = {}

      for t in targets:
          match = target_dfs[t][target_dfs[t]["作品名稱"] == title]
          cross_ids[t] = int(match.iloc[0]["ID"]) if not match.empty else ""

      # 至少有一個對應才顯示
      if any(cross_ids.values()):
          entry = {
              "ID": base_id,
              "作品名稱": title
          }

          for t, col_name in zip(targets, CROSS_MEDIA_CONFIG[media]["columns"]):
              entry[col_name] = cross_ids[t]

          rows.append(entry)

  return pd.DataFrame(rows, columns=get_cross_columns(media))

In [78]:
#查詢系統
SEARCH_COLUMNS = [
    "ID", "作品名稱", "作者", "評級", "進度",
    "狀態", "最後紀錄日期", "距離上次紀錄(天)", "類型"
]
def get_all_media_data():
  dfs = []

  for media, label in [
      ("novel", "小說"),
      ("comic", "漫畫"),
      ("anime", "動畫")
  ]:
    df = read_data(media)
    if df.empty:
        continue

    df = df.copy()
    df["類型"] = label
    dfs.append(df)

  if not dfs:
    return pd.DataFrame(columns=SEARCH_COLUMNS)

  return pd.concat(dfs, ignore_index=True)

def search_works(keyword, rating, status, min_days):
  df = get_all_media_data()

  if df.empty:
      return df

  # 關鍵字搜尋（ID / 作品 / 作者）
  if keyword:
    keyword = str(keyword).strip()
    df = df[
      df["作品名稱"].astype(str).str.contains(keyword, case=False, na=False) |
      df["作者"].astype(str).str.contains(keyword, case=False, na=False) |
      df["ID"].astype(str).str.contains(keyword, na=False)
    ]

  # 評級篩選
  if rating != "全部":
    df = df[df["評級"] == rating]

  # 狀態篩選
  if status != "全部":
    df = df[df["狀態"] == status]

  # 距離上次紀錄
  if min_days is not None:
    df = df[df["距離上次紀錄(天)"] >= min_days]

  return df[SEARCH_COLUMNS]

In [79]:
import gradio as gr

#gradio操作介面
with gr.Blocks() as app:
  gr.Markdown("追番清單")
  with gr.Tab("小說清單"): #分頁
    gr.Markdown("直接編輯清單後點擊儲存更變即可儲存!")
    refresh_novel = gr.Button("重新整理") #刷新按鈕
    with gr.Row():
      add_novel = gr.Button("新增一筆")
      del_novel_id = gr.Number(label="刪除 ID", precision=0)
      del_novel = gr.Button("刪除該筆", variant="stop")
    save_novel = gr.Button("儲存變更", variant="secondary")

    table_novel = gr.Dataframe( #表單
      value=read_data("novel"),
      headers=COLUMNS,
      interactive=True
    )

    gr.Markdown("系統偵測到以下作品存在跨媒體")
    cross_table_novel = gr.Dataframe(
        value=build_cross_media_table("novel"),
        headers=get_cross_columns("novel"),
        interactive=False
    )

    refresh_novel.click(fn=lambda: read_data("novel"), outputs=table_novel)
    add_novel.click(
        fn=lambda table: add_empty_row(table, "novel"),
        inputs=table_novel,
        outputs=table_novel
    )
    del_novel.click(
      fn=lambda rid: delete_and_refresh(rid, "novel"),
      inputs=del_novel_id,
      outputs=table_novel
    )

    save_novel.click(
        fn=lambda table: save_table(table, "novel"),
        inputs=table_novel,
        outputs=table_novel
    )

  with gr.Tab("漫畫清單"):
    gr.Markdown("直接編輯清單後點擊儲存更變即可儲存!")
    refresh_comic = gr.Button("重新整理") #刷新按鈕
    with gr.Row():
      add_comic = gr.Button("新增一筆")
      del_comic_id = gr.Number(label="刪除 ID", precision=0)
      del_comic = gr.Button("刪除該筆", variant="stop")
    save_comic = gr.Button("儲存變更", variant="secondary")

    table_comic = gr.Dataframe( #表單
      value=read_data("comic"),
      headers=COLUMNS,
      interactive=True
    )

    gr.Markdown("系統偵測到以下作品存在跨媒體")
    cross_table_comic = gr.Dataframe(
        value=build_cross_media_table("comic"),
        headers=get_cross_columns("comic"),
        interactive=False
    )

    refresh_comic.click(fn=lambda: read_data("comic"), outputs=table_comic)
    add_comic.click(
        fn=lambda table: add_empty_row(table, "comic"),
        inputs=table_comic,
        outputs=table_comic
    )
    del_comic.click(
      fn=lambda rid: delete_and_refresh(rid, "comic"),
      inputs=del_comic_id,
      outputs=table_comic
    )

    save_comic.click(
        fn=lambda table: save_table(table, "comic"),
        inputs=table_comic,
        outputs=table_comic
    )

  with gr.Tab("動畫清單"):
    gr.Markdown("直接編輯清單後點擊儲存更變即可儲存!")
    refresh_anime = gr.Button("重新整理") #刷新按鈕
    with gr.Row():
      add_anime = gr.Button("新增一筆")
      del_anime_id = gr.Number(label="刪除 ID", precision=0)
      del_anime = gr.Button("刪除該筆", variant="stop")
    save_anime = gr.Button("儲存變更", variant="secondary")

    table_anime = gr.Dataframe( #表單
      value=read_data("anime"),
      headers=COLUMNS,
      interactive=True
    )

    gr.Markdown("系統偵測到以下作品存在跨媒體")
    cross_table_anime = gr.Dataframe(
        value=build_cross_media_table("anime"),
        headers=get_cross_columns("anime"),
        interactive=False
    )

    refresh_anime.click(fn=lambda: read_data("anime"), outputs=table_anime)
    add_anime.click(
        fn=lambda table: add_empty_row(table, "anime"),
        inputs=table_anime,
        outputs=table_anime
    )
    del_anime.click(
      fn=lambda rid: delete_and_refresh(rid, "anime"),
      inputs=del_anime_id,
      outputs=table_anime
    )

    save_anime.click(
        fn=lambda table: save_table(table, "anime"),
        inputs=table_anime,
        outputs=table_anime
    )

  with gr.Tab("示警清單"):
    gr.Markdown("僅顯示「未完結」且長時間未更新的作品")

    alert_refresh = gr.Button("重新整理示警清單")

    gr.Markdown("小說示警（超過 3 個月）")
    alert_novel = gr.Dataframe(
        value=get_alert_table("novel"),
        headers=ALERT_COLUMNS,
        interactive=False
    )

    gr.Markdown("漫畫示警（超過 1 個月）")
    alert_comic = gr.Dataframe(
        value=get_alert_table("comic"),
        headers=ALERT_COLUMNS,
        interactive=False
    )

    gr.Markdown("動畫示警（超過 14 天）")
    alert_anime = gr.Dataframe(
        value=get_alert_table("anime"),
        headers=ALERT_COLUMNS,
        interactive=False
    )

    alert_refresh.click(
      fn=lambda: (
          get_alert_table("novel"),
          get_alert_table("comic"),
          get_alert_table("anime")
      ),
      outputs=[alert_novel, alert_comic, alert_anime]
    )

  with gr.Tab("查詢系統"):
    gr.Markdown("### 跨媒體作品查詢")

    with gr.Row():
      keyword = gr.Textbox(label="關鍵字（ID / 作品名稱 / 作者）")
      rating = gr.Dropdown(
        choices=["全部", "S", "A", "B", "C"],
        value="全部",
        label="評級"
      )
      status = gr.Dropdown(
        choices=["全部", "未完結", "完結"],
        value="全部",
        label="狀態"
      )
      min_days = gr.Number(
        label="距離上次紀錄 ≥ N 天",
        precision=0
      )

    search_btn = gr.Button("查詢")

    search_table = gr.Dataframe(
      headers=SEARCH_COLUMNS,
      interactive=False
    )

    search_btn.click(
      fn=search_works,
      inputs=[keyword, rating, status, min_days],
      outputs=search_table
    )

app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6792a78c0169824d04.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
